In [3]:
import gym
import seagul.envs
import numpy as np
from numpy import pi, sin, cos
import time
import torch
import torch.nn as nn
torch.set_default_dtype(torch.double)
import matplotlib.pyplot as plt

import numpy as np
from numpy import sin, cos, pi
from control import lqr

from seagul.rl.models import PPOModelActHold
from seagul.nn import MLP
from seagul.rl.run_utils import load_workspace

from scipy.integrate import solve_ivp

# dynamics from our gym environment
I2 = .33; I1 = .083
l1 = 1; l2 = 2.1
m1 = 1; m2 = 1
lc1 = .5; lc2 = 1
g = 9.8
th1 = pi/2; th2 = 0
th1d = 0; th2d = 0


B = np.array([[0],[1]])
M = np.array([[I1 + I2 + m2*l1**2 + 2*m2*l1*lc2*cos(th2), I2 + m2*l1*lc2*cos(th2)], [I2 + m2*l1*lc2*cos(th2), I2]])
C = np.array([[-2*m2*l1*lc2*sin(th2)*th2d, -m2*l1*lc2*sin(th2)*th2d], [m2*l1*lc2*sin(th2)*th1d, 0]])
G = np.array([[-m1*g*lc1*cos(th1) - m2*g*(l1*cos(th1) + lc2*cos(th1 + th2))],[-m2*g*lc2*cos(th1+ th2)]])

Alin= np.array([[0,0, 1, 0],[0, 0, 0, 1], [g*(m1*l1 + m2*l1 + m2*lc2), (m2*g*lc2), 0, 0 ],[m2*g*lc2, m2*g*lc2, 0, 0]])
Bl = np.linalg.inv(M)*B
Blin = np.array([[0,0],[0,0],Bl[0,:], Bl[1,:]])

Q = np.identity(4)
Q[0,0] = 100
Q[1,1] = 100
Q[2,2] = 10
Q[3,3] = 10

R = np.identity(2)*1
K,S,E = lqr(Alin,Blin,Q,R)
print(-K[1,:])

[[204.06707408  59.21086694  37.41566423   5.29518038]]
